In [7]:
from groq import Groq
import pandas as pd
import re
import os

In [8]:
df = pd.read_csv('../../data_acquisition/data/df_with_salary.csv')
df_with_salary_few_shot = pd.read_csv('../../data_acquisition/data/df_with_salary_few_shot.csv')

In [9]:
df.head()

,title,company,contract_type,description,min_workload,max_workload,min_salary,max_salary
0,Software Engineer .Net,ELCA Informatik AG,Festanstellung,"Über ELCA Wir sind ELCA, einer der grössten Sc...",80,100,100000,130000
1,Digital Analyst / Web Analyst,Unic AG,Festanstellung,Deine Aufgaben als Digital Analyst bei Unic ...,80,100,90000,113000
2,ICT Supporter 100% (a),Spitex Zürich,Festanstellung,Möchtest du auch etwas bewegen und deine Arbei...,100,100,78000,96000
3,KAUFMÄNNISCHES PRAKTIKUM (100%) - MIT FOKUS KU...,gebana AG,Praktikum,DEINE AUFGABEN IM KUND:INNENSERVICE – EINSATZ ...,100,100,28300,28300
4,System Engineer für Microsoft-Technologien und...,konekkt GmbH,Festanstellung,Wir präsentieren eine faszinierende Karrieremö...,80,100,95000,125000


Block 0: Few shot 5 examples with salary 
Block 1: Job title 
Block 2: company 
Block 3: workload (min workload and max workload) 
Block 4: Contract type 
Block 5: description

# Set up for Usage

In [17]:
client = Groq(
    api_key="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",
)


def extract_salaries(response):
    # General regex pattern - covers most cases
    general_pattern = re.compile(r"(\d{1,3}(?:['\s]\d{3})*)\s*(?:-|und)\s*(\d{1,3}(?:['\s]\d{3})*)\s*CHF")
    # Special regex pattern - handles "CHF" before the numbers and potential markdown
    special_pattern = re.compile(r"CHF\s*(\d{1,3}(?:[',\s]\d{3})*)\s*[-–]\s*CHF\s*(\d{1,3}(?:[',\s]\d{3})*)")

    # Try the general pattern first
    matches = general_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    # If no matches, try the special pattern
    matches = special_pattern.findall(response)
    if matches:
        min_salary, max_salary = [int(salary.replace("'", "").replace(",", "").replace(" ", "")) for salary in matches[0]]
        return min_salary, max_salary
    
    return None, None

## LLM request with Block 1 and fewshot with Block 1 and salary


In [ ]:
# change here for new prompt and message below ------------
results_file = 'data/llm_block1_fewshot.csv'
min = 'block1_min_salary_fewshot'
max = 'block1_max_salary_fewshot'
answer = 'block1_answer_fewshot'
# Setup and load initial data
if os.path.exists(results_file):
    df = pd.read_csv(results_file)
else:
    df[min] = None  # Initialize columns if starting fresh
    df[max] = None
    df[answer] = None 

# change content here for new prompt and message below ------------
def generate_query(row):
    return {
        "role": "user",
        "content": f"Gib mir nur das Gehalt im Format (\d{1,3}(?:'\d{3})*) und (\d{1,3}(?:'\d{3})) CHF für die Position {row['title']} in Zürich, Schweiz. Wenn du kein exaktes Gehalt hast gib eine Schätzung an."
    }

def generate_message(title, company, min_salary, max_salary, min_workload, max_workload, description):
    return {
        "role": "user",
        "content": f"Das Gehalt für die Position {title}  liegt zwischen {min_salary} und {max_salary}."
    }

def run_model(limit):
    start_index = df[min].last_valid_index() + 1 if df[max].last_valid_index() != None else 0
    example_messages = [generate_message(row['title'], row['company'], row['min_salary'], row['max_salary'], row['min_workload'], row['max_workload'], row['description']) for index, row in df_with_salary_few_shot.iterrows()]
    
    for index, row in df.iloc[start_index:start_index + limit].iterrows():
        query_message = generate_query(row)
        chat_completion = client.chat.completions.create(
            messages= example_messages + [query_message], # change message for new prompt ------------
            model="llama3-70b-8192",
            temperature=0.0
        )
        min_salary, max_salary = extract_salaries(chat_completion.choices[0].message.content)
        print(index)
        print(min_salary, max_salary)
        
        
        # Update DataFrame
        df.at[index, min] = min_salary
        df.at[index, max] = max_salary
        df.at[index, answer] = chat_completion.choices[0].message.content
        
    # Save updated DataFrame in chunks or fully, depending on your preference
    df.to_csv(results_file, index=False)

# Testing the function with a limited number of entries
run_model(10000)

0
90000 140000
1
None None
2
60000 90000
3
45000 60000
4
None None
5
None None
